# Understanding PyTorch

We use this notebook to study about the structure of PyTorch

In [4]:
import sys
import gym
import time
import numpy as np  

import torch  
import torch.nn            as nn
import torch.nn.functional as F
import torch.optim         as optim
from   torch.autograd import Variable

import matplotlib.pyplot   as plt


In [14]:

layer1 = nn.Linear( 1, 5 )

tmp = torch.from_numpy( np.array( [1,4,5,6,1 ] ) ).float( ).unsqueeze( 0 )
print( tmp.shape )




torch.Size([1, 5])


# Example Network
We use a three layer network

In [ ]:

class ExampleNetwork( nn.Module ):
    def __init__( self, num_inputs, num_outputs, hidden_size, learning_rate = 3e-4 ):
        super( ExampleNetwork , self).__init__()

        self.num_actions = num_inputs
        self.linear1     = nn.Linear( num_inputs, hidden_size  )
        self.linear2     = nn.Linear( hidden_size, num_inputs )

        # Using the ADAM optimizer. 
        self.optimizer   = optim.Adam( self.parameters( ), lr = learning_rate )

    def forward( self, state ):
        x = F.relu( self.linear1( state ) )
        x = F.softmax( self.linear2( x ), dim = 1 )
        return x 
    
    def get_action( self, state ):
        state = torch.from_numpy( state ).float( ).unsqueeze( 0 ) # Makes a 1 x ns tensor.

        # Forward the neural network and return the prbability distribution function
        probs = self.forward( Variable( state ) )

        # Choosing the action based on the output policy
        highest_prob_action = np.random.choice( self.num_actions, p = np.squeeze( probs.detach( ).numpy( ) ) )

        # The log-value of the probability just for the sake of policy gradient
        log_prob = torch.log( probs.squeeze( 0 )[ highest_prob_action ] )
        
        return highest_prob_action, log_prob
